In [1]:
##################################################################
# Import functions
##################################################################
import os
import pandas as pd
import numpy as np

import dash
import dash_html_components as html
import dash_core_components as dcc 
import dash_leaflet as dl
from dash_leaflet import express as dlx
from dash.dependencies import Input, Output
from shapely.geometry import shape, Point
import json

In [2]:
# Convert data to geopandas 
#import geopandas
#myshpfile = geopandas.read_file('myshpfile.shp')
#myshpfile.to_file('myJson.geojson', driver='GeoJSON')

In [3]:
##################################################################
# Load data
##################################################################

#workdirectory_data= 'C:\\Users\\nlbanni5\\Documents\\Rode Kruis\\floodcorrelation'
workdirectory_data= 'C:\\Users\\nlbrus08\\Documents\\01 Klanten\\Rode Kruis\\git_RK\\rode-kruis\\connected_areas'

os.chdir(workdirectory_data)

# Read in percentage data
df_perc_loaded = pd.read_csv('stored_data_uganda\\df_percentages_10yr_95percentile.csv').set_index('Unnamed: 0')

# Read in station locations
df_stations = pd.read_csv('stored_data_uganda\\rp_glofas_station.csv')

# Read in layers (grid, rivers and waterbodies)
with open("shapefiles/grid_layers/grid_layer_1.json", 'r') as f:
    grid = json.load(f)

with open("shapefiles/Rivers/Rivers_hydroshed_cliped_uga.json", 'r') as f:
    rivers = json.load(f)

with open("shapefiles/water_bodies/Ug_Waterbodies.json", 'r') as f:
    water_bodies = json.load(f)

with open("shapefiles/admin_boundaries/uga_adminboundaries_1.json", 'r') as f:
    admin_boundaries = json.load(f)

In [4]:
##################################################################
# Define functions
##################################################################

global select_neighbours_percentages

def select_neighbours_percentages(data_perc_overlap: pd.DataFrame = None,
                                  steps: int = 10,
                                  poi_start: str = '2.45_32.25',
                                  verbose = False,
                                  limit: int = 60):

    # Select neighbours of point of interest
    df_total_selected = data_perc_overlap[poi_start].dropna().reset_index().rename(
            columns={'Unnamed: 0': 'neighbour', poi_start: 'percentage'})
    df_total_selected['poi'] = poi_start
    
    # Select neighbour of poi with max perc
    max_percentage = df_total_selected.iloc[df_total_selected['percentage'].argmax(),1]
    print(max_percentage)
    neighbour_max = df_total_selected.iloc[df_total_selected['percentage'].argmax(),0]
    
    if max_percentage > limit: 
        df_total_selected.loc[df_total_selected.neighbour == neighbour_max, 'selected'] = 1
        
        if verbose:
            print('\nSelected neighbour: ', str(neighbour_max))
            print('Selected cell/poi in first place:', df_total_selected.loc[df_total_selected.neighbour == neighbour_max, 'poi'].item())
            print('Percentage overlap: ', round(df_total_selected.loc[df_total_selected.neighbour == neighbour_max, 'percentage'].item()))
            
        for step in range(steps):
            
            try: 
                
                df_neighbours_new = data_perc_overlap[neighbour_max].dropna().reset_index().rename(
                        columns={'Unnamed: 0': 'neighbour', neighbour_max: 'percentage'})
                df_neighbours_new['poi'] = neighbour_max

                # Only keep the new grid cells
                list_cells_already_selected = df_total_selected.neighbour.tolist() + df_total_selected.poi.unique().tolist()
                df_neighbours_new = df_neighbours_new.loc[~df_neighbours_new.neighbour.isin(list_cells_already_selected)]
                
                # Add the new select neighbours to the total dataframe
                df_total_selected = df_total_selected.append(df_neighbours_new, ignore_index = True)
                
                # Select the max neighbour
                max_percentage = df_total_selected.iloc[df_total_selected.loc[df_total_selected ['selected'] != 1]['percentage'].idxmax(),1]
                neighbour_max = df_total_selected.iloc[df_total_selected.loc[df_total_selected ['selected'] != 1]['percentage'].idxmax(),0]
                if max_percentage > limit: 
                    df_total_selected.loc[df_total_selected.neighbour == neighbour_max, 'selected'] = 1
                    
                    if verbose:
                        print('\nSelected neighbour: ', str(neighbour_max))
                        print('Selected cell/poi in first place:', df_total_selected.loc[df_total_selected.neighbour == neighbour_max, 'poi'].item())
                        print('Percentage overlap: ', round(df_total_selected.loc[df_total_selected.neighbour == neighbour_max, 'percentage'].item()))
                else: 
                    break
            
            except: 
                print('End')

    
    return(df_total_selected)

In [5]:
global get_color

def get_color(perc):
    if perc > 80:
        col = '#08519C'
    elif perc > 60:
        col = '#3182BD'
    elif perc > 40:
        col =  '#6BAED6'
    elif perc > 20:
        col = '#BDD7E7'
    else:
        col = '#EFF3FF'
    return (col)

In [6]:
global get_districts
def get_districts(df: pd.DataFrame = None, 
                  districts: json = admin_boundaries,
                  points: list = [],
                  district_list: list = []):
    
    district_list.clear()
    points.clear()
    
    for index, row in df.iterrows():
        lat = float(row['lat'])
        lon = float(row['lon'])
        point = Point(lon, lat)        
        combis_coordinates = [[ [round(lon-0.05,2), round(lat-0.05,2)], 
                                [round(lon-0.05,2), round(lat+0.05,2)], 
                                [round(lon+0.05,2), round(lat+0.05,2)],
                               [round(lon+0.05,2), round(lat-0.05,2)]
                              ]]
        points.append(combis_coordinates)

    for point in points: 
        point = shape({'type': 'Polygon', 'coordinates': point})
        for feature in districts['features']:
                polygon = shape(feature['geometry'])
                if polygon.intersects(point):
                    district_list.append(feature['properties']['ADM2_EN'])
              
    return(list(dict.fromkeys(district_list)))

In [7]:
# ## Determine data
# data = df_perc_loaded.copy()
# poi_start = '2.45_32.25'
# save = False
# name_csv = 'df_selection'+'_'+ poi_start
# path_save = 'C:\\Users\\nlbrus08\\EY\\Red Cross - 510 IbF - General\\05. Insights data\\'

# global df_selection
# # Create selection for visual
# df_selection = select_neighbours_percentages(data_perc_overlap = data,
#                                              steps = 10,
#                                              poi_start = poi_start, #'1.45_32.05',
#                                              verbose = False,
#                                              limit = 60)
# df_selection.head()

# # Select min/max lats of each and get range
# min_lat, max_lat = df_selection.neighbour.apply(lambda x: x.split('_')[0]).min(), df_selection.neighbour.apply(lambda x: x.split('_')[0]).max()
# range_lat = np.arange(round(float(min_lat),2), round(float(max_lat),2), 0.1)

# # Add column with lat and lon to df_selection
# df_selection['lat'] = df_selection.neighbour.apply(lambda x: x.split('_')[0])
# df_selection['lon'] = df_selection.neighbour.apply(lambda x: x.split('_')[1])

# # Create dataframe with latitudes as index
# df_perc = pd.DataFrame()

# # Using the for loop, add percentages and longitudes as column
# for i in df_selection.index.tolist():
#     percentage = round(df_selection.percentage[i],2)
#     lon = df_selection.lon[i]
#     lat = df_selection.lat[i]
#     df_perc.loc[lat, lon] = percentage
 
# df_perc = df_perc.sort_index(ascending=False)

# if save:
#     os.chdir(path_save)
#     df_perc.to_csv(name_csv + '.csv', sep=';', decimal=',')
#     print('saved')
    
# df_selection.head()



98.36065573770493


,neighbour,percentage,poi,selected,lat,lon
0,2.55_32.35,43.169399,2.45_32.25,NaN,2.55,32.35
1,2.55_32.25,44.262295,2.45_32.25,NaN,2.55,32.25
2,2.55_32.15,36.612022,2.45_32.25,NaN,2.55,32.15
3,2.45_32.35,61.202186,2.45_32.25,1.0,2.45,32.35
4,2.45_32.15,48.087432,2.45_32.25,NaN,2.45,32.15


In [ ]:
##################################################################
# Webapp / interactive map
##################################################################

global steps
steps = 10
global min_perc
min_perc = 60
global districts_df 
districts_df = {}

# data
data = df_perc_loaded.copy()
stations_uganda = df_stations[df_stations.Country=='Uganda']

# Settings
options = dict(hoverStyle=dict(weight = 5, color = '#666', dashArray = ''), zoomToBoundsOnClick=False)

# Create color bar
marks = [20,40,60,80]
colorscale = ['#EFF3FF', '#BDD7E7', '#6BAED6', '#3182BD', '#08519C']
ctg = ["{}+".format(mark, marks[i + 1]) for i, mark in enumerate(marks[:-1])] + ["{}+".format(marks[-1])]
color_bar = dlx.categorical_colorbar(categories = ctg, colorscale=colorscale, width=300, height=30, position="bottomleft")

# Bounds
bounds_uganda = [[-1.50, 29.50], [4.30, 35.10]]

# Function
def get_style(feature):
    return dict(fill = False, weight=1, opacity=1, color='white', dashArray='3', fillOpacity=0.1)
def get_style_grey(feature):
    return dict(fill = False, weight=1, opacity=0.8, color='black', dashArray='3', fillOpacity=0.1)

river_coords = []
for river in rivers['features']:
     river_coords.append(river['geometry']['coordinates'])

coords_list = []
for coords in river_coords: 
    points = []
    for point in coords: 
        rev_point = point[::-1]
        points.append(rev_point)
    coords_list.append(points)

# Transform gridded data in right style
grid_geo = dlx.geojson(grid, id="grid_geojson", defaultOptions=options, style = get_style)
districts_geo = dlx.geojson(admin_boundaries, id="districts_geojson", defaultOptions=options, style = get_style_grey)
    
# Create: spatial file based on river geojson 
grid_rivers = dl.Polyline(positions = coords_list, weight = 1, fillColor = 'blue', fillOpacity = 0.7)

# Stations
marker_stations = [dl.Marker(position=[row['lat'], row['lon']], children=dl.Tooltip(row['ID'])) for i, 
                   row in stations_uganda.iterrows()]
# Create app
app = dash.Dash(prevent_initial_callbacks=True)

app.layout = html.Div([
                    html.H1('Interactive map that shows the relations between areas'),
                    html.Div([  
                        html.Div(
                            html.H4('Steps iterative process'), 
                            style={'width': '30%', 'display': 'inline-block'}),
                        html.Div(
                            html.H4('Minimal percentage of overlapping extreme days'), 
                            style={'width': '30%', 'display': 'inline-block'})
                    ]),
                    html.Div([  
                        html.Div(    
                        dcc.Slider(
                            id='slider_steps',
                            min=5,
                            max=30,
                            step=1,
                            value=steps,
                            dots=True,
                            marks={
                                5: '5',
                                10: '10',
                                15: '15',
                                20: '20',
                                25: '25',
                                30: '30'
                            })
                            ,style={'width': '30%', 'display': 'inline-block'}),               
                        html.Div(
                        dcc.Slider(
                            id='slider_min_perc',
                            min=0,
                            max=100,
                            step=5,
                            value=min_perc,
                            dots=True,
                            marks={
                                0 : '0%',
                                20 : '20%',
                                40 : '40%',
                                60 : '60%',
                                80: '80%',
                                100: '100%'
                            })
                        ,style={'width': '30%', 'display': 'inline-block'}),
                        ]),
                        html.Div([
                        dcc.Checklist(id="checklist_stations",
                                     options=[{'label': 'Show stations', 'value':'stations'}],
                                      value = False
                                               ), 
                        ]),
                        html.Hr(),
                        html.Div(id='print_districts'), # style={'width': '40%', 'display': 'inline-block'})
                        html.Hr(),
                        html.Div([
                                dl.Map(id="map",
                                # Order of this list determines overlay order 
                                children = [grid_rivers,
                                           #grid_water_bodies,
                                           dl.TileLayer(),                                        
                                           dl.LayerGroup(id="layer"), 
                                           dl.LayerGroup(id='rec_neighbours'),    
                                           dl.LayerGroup(id='rec_pois'),
                                           dl.LayerGroup(id='marker_stations'),
                                           color_bar,
                                           grid_geo,
                                           districts_geo
                                          ],
                                   zoom=7, center = [1.40, 32.30],
                               doubleClickZoom = True,               
                               maxBounds  = bounds_uganda,
                               style={'width': '60%', 'height': '80vh', "display": "inline-block"})                               
                           ])
                        
        ])

@app.callback(Output("marker_stations", "children"), 
              [Input("checklist_stations", 
                     "value")])
def update_output(value):
    print(value)
    if 'stations' in value:
        print('check')
        return [dl.Marker(position=[row['lat'], row['lon']], children=dl.Tooltip(row['ID'])) for i, 
                               row in stations_uganda.iterrows()]
    else:
        print('none')
        return None

@app.callback(Output("layer", "children"), 
              [Input("map", "click_lat_lng")])
def map_click(click_lat_lng):
    return [dl.Marker(position=click_lat_lng, children=dl.Tooltip('start'+"({:.2f}, {:.2f})".format(*click_lat_lng) )) ]

@app.callback(Output("print_districts", "children"), 
              [Input("map", "click_lat_lng"),
              Input("slider_steps", "value"),
              Input("slider_min_perc", "value")])
def map_click(click_lat_lng, steps, min_perc):
    lat_min = np.floor(click_lat_lng[0]*10)/10
    lat_max = np.ceil(click_lat_lng[0]*10)/10
    lon_min = np.floor(click_lat_lng[1]*10)/10
    lon_max = np.ceil(click_lat_lng[1]*10)/10
    
    poi_start = str(round((lat_min + 0.05),2)) + '_' + str(round((lon_min+0.05),2))
    
    df_output = select_neighbours_percentages(data_perc_overlap = data, poi_start = poi_start, steps = steps, limit = int(min_perc))
    new_pois = df_output.poi.unique().tolist()
    
    # Add column with lat and lon to df_selection
    lat = [x.split('_')[0] for x in new_pois]
    lon = [x.split('_')[1] for x in new_pois]

    df = pd.DataFrame(columns=["lat", "lon"], data=np.column_stack((lat,lon)))
    districts_df = get_districts(df=df, districts = admin_boundaries)
    
    if len(districts_df) > 0:
        return 'Disctricts connected to the selected cell are: ' + str(districts_df).strip('[]')
    else:
        return 'No disctricts connected to the selected cell'

@app.callback(Output("rec_pois", "children"), 
              [Input("map", "click_lat_lng"),
              Input("slider_steps", "value"),
              Input("slider_min_perc", "value")])
def map_click(click_lat_lng, steps, min_perc):
    lat_min = np.floor(click_lat_lng[0]*10)/10
    lat_max = np.ceil(click_lat_lng[0]*10)/10
    lon_min = np.floor(click_lat_lng[1]*10)/10
    lon_max = np.ceil(click_lat_lng[1]*10)/10
    
    poi_start = str(round((lat_min + 0.05),2)) + '_' + str(round((lon_min+0.05),2))
    
    df_output = select_neighbours_percentages(data_perc_overlap = data, poi_start = poi_start, steps = steps, limit = int(min_perc))
    new_pois = df_output.poi.unique().tolist()
    
    # Add column with lat and lon to df_selection
    lat = [x.split('_')[0] for x in new_pois]
    lon = [x.split('_')[1] for x in new_pois]

    df = pd.DataFrame(columns=["lat", "lon"], data=np.column_stack((lat,lon)))
    
    districts_df = get_districts(df, admin_boundaries)
    
    return [dl.Rectangle(bounds=[[round(float(row['lat'])-0.05,2), round(float(row['lon'])-0.05,2)],
                                 [round(float(row['lat'])+0.05,2), round(float(row['lon'])+0.05,2)]],
                        fill = False, opacity = .65, color = 'blue') for i, row in df.iterrows()] 

@app.callback(Output("rec_neighbours", "children"), 
              [Input("map", "click_lat_lng"),
              Input("slider_steps", "value"),
              Input("slider_min_perc", "value")])

def map_click(click_lat_lng, steps, min_perc):
    lat_min = np.floor(click_lat_lng[0]*10)/10
    lat_max = np.ceil(click_lat_lng[0]*10)/10
    lon_min = np.floor(click_lat_lng[1]*10)/10
    lon_max = np.ceil(click_lat_lng[1]*10)/10
    
    poi_start = str(round((lat_min + 0.05),2)) + '_' + str(round((lon_min+0.05),2))
    df_output = select_neighbours_percentages(data_perc_overlap = data, poi_start = poi_start, steps = steps, limit = int(min_perc))
    new_nbrs = df_output.neighbour.unique().tolist()
    
    # Add column with lat and lon to df_selection
    lat = [x.split('_')[0] for x in new_nbrs]
    lon = [x.split('_')[1] for x in new_nbrs]
    perc = [str(round(x)) for x in df_output.percentage.tolist()]

    df_nbr = pd.DataFrame(columns=["lat", "lon", "perc"], data=np.column_stack((lat,lon,perc)))
    
    return [dl.Rectangle(bounds=[[round(float(row['lat'])-0.05,2), round(float(row['lon'])-0.05,2)],
                                 [round(float(row['lat'])+0.05,2), round(float(row['lon'])+0.05,2)]],
                        stroke = False, 
                        fillColor= get_color(float(row['perc'])),
                        weight = 1, 
                        opacity = 1,
                        fillOpacity = .7,
                        children=dl.Tooltip(row['perc']+'%')
                        ) for i, row in df_nbr.iterrows()] 

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jul/2020 17:15:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:15:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:15:15] "GET /_favicon.ico?v=1.13.4 HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:15:16] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:15:21] "POST /_dash-update-component HTTP/1.1" 200 -


95.08196721311475
95.08196721311475
95.08196721311475


127.0.0.1 - - [23/Jul/2020 17:15:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:15:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:15:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:15:26] "POST /_dash-update-component HTTP/1.1" 200 -


99.4535519125683
99.4535519125683
99.4535519125683


127.0.0.1 - - [23/Jul/2020 17:15:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:15:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:15:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:29:45] "POST /_dash-update-component HTTP/1.1" 200 -


100.0100.0

100.0


127.0.0.1 - - [23/Jul/2020 17:29:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:29:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jul/2020 17:29:46] "POST /_dash-update-component HTTP/1.1" 200 -
